In [1]:
import argparse
import pickle
from model import NLG
from data_engine import DataEngine
from text_token import _UNK, _PAD, _BOS, _EOS
import torch
import torch.nn as nn
import numpy as np
import os
from utils import print_config, add_path
from model_utils import get_embeddings
from argument import define_arguments
from utils import get_time


In [14]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn.utils import clip_grad_norm

import random
import numpy as np
import os

from module import EncoderRNN
from module import DecoderRNN
# from utils import single_BLEU, BLEU, single_ROUGE, ROUGE, best_ROUGE, print_time_info, check_dir, print_curriculum_status
from utils import *
from text_token import _UNK, _PAD, _BOS, _EOS
from model_utils import collate_fn, build_optimizer
from logger import Logger

from tqdm import tqdm


In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn.utils import clip_grad_norm

import random
import numpy as np
import os

from module import EncoderRNN
from module import DecoderRNN
# from utils import single_BLEU, BLEU, single_ROUGE, ROUGE, best_ROUGE, print_time_info, check_dir, print_curriculum_status
from utils import *
from text_token import _UNK, _PAD, _BOS, _EOS
from model_utils import collate_fn, build_optimizer
from logger import Logger

from tqdm import tqdm

In [11]:
import sys

sys.argv=["train.py" ,"--data_dir=./data/"]

In [12]:
_, args = define_arguments()

args = add_path(args)
if args.verbose_level > 0:
    print_config(args)


PATH:
	Data directory: ./data/E2ENLG
	Embeddings directory: ./data/GloVe
	Model directory: ./data/model_slt
	Log directory: ./data/log_slt
	Processed train data file: ./data/E2ENLG_train_data.pkl
	Processed valid data file: ./data/E2ENLG_valid_data.pkl
	Processed vocab file: ./data/E2ENLG_vocab.pkl
DATA:
	Dataset: E2ENLG
	Regenerate dataset: False
	Fold attributes: True
	Vocab size: 500
	Use pretrained embeddings: False
MODEL:
	RNN cell: GRU
	Decoder layers: 4
	RNN layers of encoder: 1
	RNN layers of decoder: 1
	Hidden size of encoder RNN: 200
	Hidden size of decoder RNN: 100
	Encoder embedding layer: False
	Decoder embedding dimension: 50
	Attention method: none
	Bidrectional RNN: True
	Batch normalization: False
TRAINING:
	Training epochs: 20
	Batch size: 32
	Encoder optimizer: Adam
	Encoder learning_rate: 0.001
	Decoder optimizer: Adam
	Decoder learning rate: 0.001
	Inner teacher forcing ratio: 0.5
	Inter teacher forcing ratio: 0.5
	Inner teacher forcing decay rate: 0.9
	Inter teac

In [13]:

use_cuda = torch.cuda.is_available()
train_data_engine = DataEngine(
    data_dir=args.data_dir,
    dataset=args.dataset,
    save_path=args.train_data_file,
    vocab_path=args.vocab_file,
    is_spacy=args.is_spacy,
    is_lemma=args.is_lemma,
    fold_attr=args.fold_attr,
    use_punct=args.use_punct,
    vocab_size=args.vocab_size,
    n_layers=args.n_layers,
    en_max_length=(args.en_max_length if args.en_max_length != -1 else None),
    de_max_length=(args.de_max_length if args.de_max_length != -1 else None),
    regen=args.regen,
    train=True
)

[2019-04-28 02:30:55] Use Spacy as the parser
[2019-04-28 02:30:55] Read vocab data from ./data/E2ENLG_vocab.pkl
[2019-04-28 02:30:55] Load the data from ./data/E2ENLG_train_data.pkl
[2019-04-28 02:30:55] Shrink vocab size to 500 + _UNK, _BOS, _EOS, _PAD


In [15]:
train_data_loader = DataLoader(
        train_data_engine,
        batch_size=16,
        shuffle=True,
        num_workers=1,
        drop_last=True,
        collate_fn=collate_fn,
        pin_memory=True)

In [24]:
one_batch=next(train_data_loader.__iter__())

# fish=[]
# for b_idx, batch in enumerate(tqdm(train_data_loader)):
#     fish.append(batch)
#     break
# caught_batch=fish[0]

In [26]:
len(one_batch)

5

In [27]:
raw_encoder_input, decoder_labels, de_lengths, refs, sf_data = one_batch

In [47]:
encoder_input = [
        train_data_engine.tokenizer.untokenize(sent, sf_data[idx], is_token=True)
        for idx, sent in enumerate(raw_encoder_input)]

In [56]:
#train_data_engine.sf_data

In [60]:
decoder_labels[2].shape

(16, 22)